In [1]:
print("start import")
import time
time1 = time.time()
import torch
from torch_geometric.data import Data # 从torch_geometric.data导入Data类，用于表示图数据
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
import torch.nn as nn

time2 = time.time()
print("import used time: ", time2 - time1)
print(f"当前时间: {time.strftime('%m-%d %H:%M:%S', time.localtime())}")

start import
import used time:  16.681838989257812
当前时间: 11-27 23:16:13


In [6]:
#构造图数据集（3个图形）
# 三角形图（3节点，3边）
triangle = Data(
	x=torch.eye(3, dtype=torch.float),  # 3节点，3维特征（one-hot编码）
	# x=torch.tensor([
	#     [1.0, 3],  # 节点0的特征: 原始特征+附加特征。第一个特征没用。
	#     [1.0, 3],  # 节点1的特征: 原始特征+附加特征
	#     [1.0, 3]   # 节点2的特征: 原始特征+附加特征
	# ]),  # 每个节点2维特征
	edge_index=torch.tensor([[0, 1, 1, 2, 2, 0],
							 [1, 0, 2, 1, 0, 2]], dtype=torch.long), # 3边，2维边索引
	y=torch.tensor([0])  # 类别0
)

# 正方形图（4节点，4边）
square = Data(
	x=torch.eye(4, dtype=torch.float),  #one-hot编码
	# x=torch.tensor([
	#     [1.0, 4],  # 节点0的特征: 原始特征+附加特征
	#     [1.0, 4],
	#     [1.0, 4],
	#     [1.0, 4]
	# ]),  
	edge_index=torch.tensor([[0, 1, 1, 2, 2, 3, 3, 0],
							 [1, 0, 2, 1, 3, 2, 0, 3]], dtype=torch.long),
	y=torch.tensor([1])  # 类别1
)

# 五边形图（5节点，5边）
pentagon = Data(
	x=torch.eye(5, dtype=torch.float),  #one-hot编码
	# x=torch.tensor([
	#     [1.0, 5],  # 节点0的特征: 原始特征+附加特征
	#     [1.0, 5],
	#     [1.0, 5],
	#     [1.0, 5],
	#     [1.0, 5]
	# ]),  
	edge_index=torch.tensor([
		[0, 1, 1, 2, 2, 3, 3, 4, 4, 0],
		[1, 0, 2, 1, 3, 2, 4, 3, 0, 4]
	], dtype=torch.long),
	y=torch.tensor([2])  # 类别2
)

print("construct dataset")
# 构建数据集
# dataset = [triangle, square, pentagon]
# loader = DataLoader(dataset, batch_size=1, shuffle=True)

# 定义一个函数，用0填充节点特征
def pad_node_features(data, target_dim=5):
	# 原始特征维度
	orig_dim = data.x.size(1) # 原始特征维度
	if orig_dim < target_dim:
		pad_size = target_dim - orig_dim
		padding = torch.zeros((data.x.size(0), pad_size), dtype=torch.float) # 用0填充缺失维度
		data.x = torch.cat([data.x, padding], dim=1) # 填充节点特征
	return data

# 定义一个函数，添加节点掩码
# def add_node_mask(data):    
#     data.node_mask = torch.ones(data.x.size(0), dtype=torch.bool)
#     return data

max_dim = 5
# dataset = [pad_node_features(add_node_mask(graph), target_dim=max_dim)
dataset = [pad_node_features(graph, target_dim=max_dim) # 填充节点特征
			for graph in [triangle, square, pentagon]]
loader = DataLoader(dataset, batch_size=1, shuffle=True)
#输出各图的节点特征和掩码
for graph in dataset:
	print(f"类别: {graph.y.item()}, 节点特征: {graph.x}")#, 节点掩码: {graph.node_mask}")
print(f"当前时间: {time.strftime('%m-%d %H:%M:%S', time.localtime())}")

construct dataset
类别: 0, 节点特征: tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])
类别: 1, 节点特征: tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])
类别: 2, 节点特征: tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
当前时间: 11-18 22:52:44


In [7]:
#定义图神经网络模型（GCN）
class GNN(nn.Module):
	def __init__(self):
		super().__init__()
		self.conv1 = GCNConv(5, 16)  # 输入层
		self.conv2 = GCNConv(16, 32) # 隐藏层
		self.fc = nn.Linear(32, 3)  # 输出3类

	def forward(self, data):
		x, edge_index, batch = data.x, data.edge_index, data.batch
		x = F.relu(self.conv1(x, edge_index)) # 输入层激活
		x = F.relu(self.conv2(x, edge_index)) # 隐藏层激活
		x = global_mean_pool(x, batch)  # 图级聚合
		return self.fc(x) # 输出层

In [8]:
from ModelTrainerClass import ModelTrainerClass
from BTNHGV2ParameterClass import BTNHGV2ParameterClass
import sys
import importlib
importlib.reload(sys.modules['ModelTrainerClass'])
importlib.reload(sys.modules['BTNHGV2ParameterClass'])
from BTNHGV2ParameterClass import BTNHGV2ParameterClass
from ModelTrainerClass import ModelTrainerClass
gmodel=GNN()
trainer=ModelTrainerClass(model=gmodel, train_loader=loader, test_loader=loader,
						lr=BTNHGV2ParameterClass.lr, patience=BTNHGV2ParameterClass.patience)
trainer.run()
trainer.test()


using device: cuda
Epoch 010 | Loss: 1.0361
Epoch 020 | Loss: 0.7161
Epoch 030 | Loss: 0.3467
Epoch 040 | Loss: 0.0922
Epoch 050 | Loss: 0.0213
Epoch 060 | Loss: 0.0099
Epoch 070 | Loss: 0.0061
Epoch 080 | Loss: 0.0043
Epoch 090 | Loss: 0.0030
Epoch 100 | Loss: 0.0019
训练完成, epoch : 100, loss: 0.0019
Accuracy: 1.0000


1.0